In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
pd.set_option("max_columns", 200)



for y in ['20200512', '20200513', '20200514', '20200515']:
    print('----------------------------------------------------------------')
    print(y)
    
    readPath = '\\\\192.168.10.30\\Kevin_zhenyu\\temp\\GTA\\' + y + '\\SZSEL2\\STOCK\\ORDER\\***\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs < 4000) | (dateLs > 300000)]
    OrderLogSZ1 = []
    for i in dataPathLs:
        df = pd.read_csv(i)
        OrderLogSZ1 += [df]
    OrderLogSZ1 = pd.concat(OrderLogSZ1).reset_index(drop=True)
    OrderLogSZ1 = OrderLogSZ1.rename(columns={"OrderPrice":"Price", "OrderVolume":"OrderQty", "SecurityID":"xx", "Symbol":"SecurityID", 
                                          "OrderCode":"Side", "RecID":"ApplSeqNum"})

    OrderLogSZ1["Price"] = OrderLogSZ1["Price"] * 10000
    OrderLogSZ1["Price"] = OrderLogSZ1["Price"].round(0)
    OrderLogSZ1["TransactTime"] = OrderLogSZ1["TradingTime"].astype(str).str[11:13] + OrderLogSZ1["TradingTime"].astype(str).str[14:16] + \
    OrderLogSZ1["TradingTime"].astype(str).str[17:19] + OrderLogSZ1["TradingTime"].astype(str).str[20:]
    OrderLogSZ1["TransactTime"] = OrderLogSZ1["TransactTime"].astype(int)

    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdOrderLog_*'))[0]
    OrderLogSZ = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
    OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
    OrderLogSZ1["OrderType"] = OrderLogSZ1["OrderType"].apply(lambda x: str(x))

    sl = list(set(OrderLogSZ["SecurityID"].unique()) & set(OrderLogSZ1['SecurityID'].unique()))
    OrderLogSZ = OrderLogSZ[OrderLogSZ["SecurityID"].isin(sl)]
    OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["SecurityID"].isin(sl)]
    print(len(OrderLogSZ["SecurityID"].unique()))
    print(len(OrderLogSZ1["SecurityID"].unique()))
    
    print('----------------------------------------------------------------')
    print('SZ order data:')
    
    columns = ["ApplSeqNum", "TransactTime","Side",'OrderType', 'Price', 'OrderQty', "SecurityID"]
    ree = pd.merge(OrderLogSZ, OrderLogSZ1, on=columns, how="outer", validate='one_to_one')
    n1 = ree["sequenceNo"].count()
    n2 = ree["xx"].count()
    len1 = len(ree)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(ree[np.isnan(ree["xx"])])
        print(len(ree[np.isnan(ree["xx"])]))
        print(np.sort(ree[np.isnan(ree["xx"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["xx"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["xx"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("test is complete, baseline is not complete:")
        display(ree[np.isnan(ree["sequenceNo"])])
        print(np.sort(ree[np.isnan(ree["sequenceNo"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo"])]["SecurityID"].unique())
        display(n2-n1)
    del OrderLogSZ
    del OrderLogSZ1
    
    
    
    
    
    readPath = '\\\\192.168.10.30\\Kevin_zhenyu\\temp\\GTA\\' + y + '\\SSEL2\\STOCK\\TRADE\\***\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs >= 600000) & (dateLs <= 700000)]
    SH1 = []
    for i in dataPathLs:
        df = pd.read_csv(i)
        SH1 += [df]
    SH1 = pd.concat(SH1).reset_index(drop=True)

    SH1 = SH1.rename(columns={"SecurityID":"xx", "BuySellFlag":"TradeBSFlag", "function_code":"ExecType",
                             "trade_price":"TradePrice", "TradeVolume":"TradeQty", "SellRecID":"OfferApplSeqNum", 
                              "BuyRecID":"BidApplSeqNum", "TradeAmount":"TradeMoney", "RecID":"ApplSeqNum"})
    SH1["TransactTime"] = SH1["TradingTime"].astype(str).str[11:13] + SH1["TradingTime"].astype(str).str[14:16] + \
    SH1["TradingTime"].astype(str).str[17:19] + SH1["TradingTime"].astype(str).str[20:]
    SH1["TransactTime"] = SH1["TransactTime"].astype(int)
    SH1["SecurityID"] = SH1["Symbol"]
    SH1["ExecType"] = 'F'
    SH1["TradePrice"] = SH1["TradePrice"] * 10000
    SH1["TradePrice"] = SH1["TradePrice"].round(0)
    SH1["TradeMoney"] = SH1["TradeMoney"] * 10000
    SH1["TradeMoney"] = SH1["TradeMoney"].round(0)

    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdTradeLog_*'))[0]
    SH = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    sl = list(set(SH["SecurityID"].unique()) & set(SH1['SecurityID'].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
    print(len(SH["SecurityID"].unique()))
    print(len(SH1["SecurityID"].unique()))

    print(SH1.columns)
    
    print('----------------------------------------------------------------')
    print('SH trade data:')
    
    SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
    SH1["ExecType"] = SH1["ExecType"].apply(lambda x: str(x))
    columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum"]
    n1 = len(SH["SecurityID"].unique())
    n2 = len(SH1["SecurityID"].unique())
    print(n1)
    print(n2)
    if n1 > n2:
        SH = SH[SH["SecurityID"].isin(SH1["SecurityID"].unique())]
    if n2 > n1:
        SH1 = SH1[SH1["SecurityID"].isin(SH["SecurityID"].unique())]
    re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo"].count()
    n2 = re["xx"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["xx"])])
        print(len(re[np.isnan(re["xx"])]))
        print(np.sort(re[np.isnan(re["xx"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["xx"])]["SecurityID"].unique()))
        print(re[np.isnan(re["xx"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo"])])
        print(np.sort(re[np.isnan(re["sequenceNo"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique())
        display(n2-n1)
    del SH
    del SH1
    
    
    
    
    
    
    
    readPath = '\\\\192.168.10.30\\Kevin_zhenyu\\temp\\GTA\\' + y + '\\SZSEL2\\STOCK\\TRADE\\***\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs < 4000) | (dateLs > 300000)]
    TradeLogSZ1 = []
    for i in dataPathLs:
        df = pd.read_csv(i)
        TradeLogSZ1 += [df]
    TradeLogSZ1 = pd.concat(TradeLogSZ1).reset_index(drop=True)

    TradeLogSZ1 = TradeLogSZ1.rename(columns={"SecurityID":"xx", "function_code":"ExecType",
                             "trade_price":"TradePrice", "TradeVolume":"TradeQty", "SellOrderID":"OfferApplSeqNum", 
                              "BuyOrderID":"BidApplSeqNum", "RecID":"ApplSeqNum", "TradeType":"ExecType"})
    TradeLogSZ1["TransactTime"] = TradeLogSZ1["TradingTime"].astype(str).str[11:13] + TradeLogSZ1["TradingTime"].astype(str).str[14:16] + \
    TradeLogSZ1["TradingTime"].astype(str).str[17:19] + TradeLogSZ1["TradingTime"].astype(str).str[20:]
    TradeLogSZ1["TransactTime"] = TradeLogSZ1["TransactTime"].astype(int)
    TradeLogSZ1["SecurityID"] = TradeLogSZ1["Symbol"]
    TradeLogSZ1["TradePrice"] = TradeLogSZ1["TradePrice"] * 10000
    TradeLogSZ1["TradePrice"] = TradeLogSZ1["TradePrice"].round(0)
    TradeLogSZ1["TradeMoney"] = TradeLogSZ1["TradePrice"] * TradeLogSZ1["TradeQty"]
    TradeLogSZ1["TradeBSFlag"] = np.where(TradeLogSZ1["SymbolSource"] == 102, 'N', 'N')
    
    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdTradeLog_*'))[0]

    TradeLogSZ = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    sl = list(set(TradeLogSZ["SecurityID"].unique()) & set(TradeLogSZ1['SecurityID'].unique()))
    TradeLogSZ = TradeLogSZ[TradeLogSZ["SecurityID"].isin(sl)]
    TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1["SecurityID"].isin(sl)]
    print(len(TradeLogSZ["SecurityID"].unique()))
    print(len(TradeLogSZ1["SecurityID"].unique()))

    print(TradeLogSZ1.columns)
    
    
    print('----------------------------------------------------------------')
    print('SZ trade data:')
    
    TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
    TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

    columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
    re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo"].count()
    n2 = re["xx"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["xx"])])
        print(len(re[np.isnan(re["xx"])]))
        print(np.sort(re[np.isnan(re["xx"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["xx"])]["SecurityID"].unique()))
        print(re[np.isnan(re["xx"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo"])])
        print(np.sort(re[np.isnan(re["sequenceNo"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique())
        display(n2-n1)
    del TradeLogSZ
    del TradeLogSZ1